# And now... PostgreSQL!

In [ ]:
from uuid import uuid4
from enum import Enum

from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

from sqlalchemy.dialects import postgresql

engine = create_engine("postgresql://alchemist@alchemist_db/alchemist", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()


class MemberType(Enum):
    FREE = 'free'
    PAYING = 'paying'
    
    @classmethod
    def as_list(cls):
        return [e.value for e in cls]

class User(Base):
    __tablename__ = 'users'
    
    id = Column(postgresql.UUID(as_uuid=True), primary_key=True, default=uuid4)
    name = Column(String)
    fullname = Column(String)
    
    shipping_address = Column(postgresql.JSONB)
    
    membership = Column(postgresql.ENUM(*MemberType.as_list(), name='membership'), 
                        default=MemberType.FREE.value)
    
    def __repr__(self):
        return '<User(name={}, fullname={}, shipping_address={}, membership={})>'.format(
            self.name, self.fullname, self.shipping_address, self.membership)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
seven = User(name='seven', fullname='Seven of Nine')
seven.shipping_address = {'zip': '10000', 'street1': 'Preradovićeva 18'}

In [ ]:
session.add(seven)
session.commit()

In [ ]:
from_db = session.query(User).filter_by(name='seven').one()
print(from_db)

In [ ]:
from_db.shipping_address['zip']
# SELECT shipping_address->zip' AS zip FROM users

In [ ]:
seven is from_db

In [ ]:
seven.membership = 'bb'

In [ ]:
session.dirty

In [ ]:
try:
    session.commit()
except Exception as e:
    session.rollback()

In [ ]:
seven